In [10]:
import pandas as pd

In [11]:
datas = pd.read_csv('./DATA/videos/대암씨_로보토미_20230404.csv')
#YoutubeDATA\DATA\videos\디글_호구들의감빵생활 레전드_20230401.csv

# calculate the date difference between rows
datas['date_diff'] = pd.to_datetime(datas['date'], format='%Y-%m-%dT%H:%M:%SZ').diff().apply(lambda x: x.total_seconds() / 3600)

# set the first date_diff value to 0
datas.loc[datas.index[0], 'date_diff'] = 0

datas['view_per_first'] = datas['views'] / datas.loc[datas.index[0], 'views']
datas['likes_per_first'] = datas['likes'] / datas.loc[datas.index[0], 'likes']
datas['likes_per_view'] = datas['likes'] / datas.loc[datas.index[0], 'views']
datas.drop(['Unnamed: 0'],axis=1,inplace=True)

datas.head()
datas.to_csv('test2.csv')

In [8]:
# read the CSV file into a Pandas DataFrame
df = pd.read_csv('./DATA/PopularRightNow/PRN_korea_20230402.csv')

# create a new DataFrame by exploding the 'tags' column
tags_df = df.explode('tags')

# group by 'tags' and aggregate the sum of 'views' and count of 'tags'
result = tags_df.groupby('tags').agg({'views': 'sum', 'tags': 'count'})

# rename the columns to more descriptive names
result = result.rename(columns={'tags': 'tag_count', 'views': 'total_views'})

# sort the DataFrame by 'total_views' in descending order
result = result.sort_values('total_views', ascending=False)


In [24]:
import pandas as pd
import ast
import numpy as np


# parse the "tags" column as a list of strings
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if x != 'Nan' else np.nan)

# drop rows with NaN values in the "tags" column
df = df.dropna(subset=['tags'])

# create a list of all tags in the DataFrame
tag_list = [tag for sublist in df['tags'] for tag in sublist]

# create a dictionary to store the value counts
value_counts = {}

# iterate over each value in the tag list
for value in tag_list:
    # add the value to the dictionary with a count of 1 if it does not already exist, or increment its count if it does
    value_counts[value] = value_counts.get(value, 0) + 1

# create a DataFrame from the dictionary of value counts
df_counts = pd.DataFrame(list(value_counts.items()), columns=['tag', 'count'])

# print the result
print(df_counts)


                   tag  count
0     YG Entertainment      2
1                   YG      3
2                  와이지      2
3                K-pop      2
4            BLACKPINK      1
...                ...    ...
2948               다이닝      1
2949            미쉐린가이드      1
2950               미쉐린      1
2951          michelin      1
2952             seoul      1

[2953 rows x 2 columns]


In [21]:
import ast

my_list = ast.literal_eval(df['tags'][0])

my_list

['YG Entertainment',
 'YG',
 '와이지',
 'K-pop',
 'BLACKPINK',
 '블랙핑크',
 '블핑',
 '제니',
 '로제',
 '리사',
 '지수',
 'LISA',
 'JISOO',
 'JENNIE',
 'ROSÉ',
 'BLINK',
 '블링크',
 '지수 꽃',
 'JISOO FLOWER',
 'FLOWER',
 'All Eyes On Me',
 'JISOO All Eyes On Me',
 '지수 All Eyes On Me',
 'JISOO ME',
 'ME',
 '지수 FLOWER',
 'JISOO 꽃']